In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Libraries imported.


In [3]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [4]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

boston_data = pd.read_csv(body)
boston_data.head()


,Neighborhood,Latitude,Longitude
0,Cambridge,42.373611,-71.110558
1,Boston,42.361145,-71.057083
2,Brighton,42.347800,-71.156600
3,Jamaica Plain,42.312600,-71.111500


Boston dataset was not available, it had to create.

In [40]:
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

SyntaxError: invalid syntax (<ipython-input-40-5018b6b36362>, line 1)

Client id and Client secret removed

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    LIMIT=100
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [9]:
boston_venues = getNearbyVenues(names=boston_data['Neighborhood'],
                                   latitudes=boston_data['Latitude'],
                                   longitudes=boston_data['Longitude']
                                  )

Cambridge
Boston
Brighton
Jamaica Plain


In [12]:
print(boston_venues.shape)
boston_venues.head()

(232, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cambridge,42.373611,-71.110558,Darwin's Ltd.,42.374928,-71.109394,Sandwich Place
1,Cambridge,42.373611,-71.110558,Harvard Art Museums,42.374206,-71.114368,Art Museum
2,Cambridge,42.373611,-71.110558,Skenderian Apothecary,42.374841,-71.108921,Pharmacy
3,Cambridge,42.373611,-71.110558,Cafe Sushi,42.370616,-71.113385,Sushi Restaurant
4,Cambridge,42.373611,-71.110558,Sanders Theatre,42.375835,-71.114786,Theater


In [13]:
boston_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Boston,100,100,100,100,100,100
Brighton,46,46,46,46,46,46
Cambridge,44,44,44,44,44,44
Jamaica Plain,42,42,42,42,42,42


We can see, only Boston central has vast oppornity of services

In [14]:
# one hot encoding
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['Neighborhood'] = boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Beer Garden,Belgian Restaurant,Bookstore,Boutique,Burger Joint,Burrito Place,Bus Station,Café,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Coffee Shop,College Theater,Comedy Club,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Dry Cleaner,Falafel Restaurant,Farmers Market,Fish Market,Food & Drink Shop,Food Truck,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym / Fitness Center,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Lingerie Store,Liquor Store,Mac & Cheese Joint,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Neighborhood,New American Restaurant,Nightclub,Noodle House,Park,Pastry Shop,Pharmacy,Pilates Studio,Pizza Place,Plaza,Pub,Ramen Restaurant,Record Shop,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Ski Chalet,Smoke Shop,Spa,Steakhouse,Street Food Gathering,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Train,Vegetarian / Vegan Restaurant,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cambridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cambridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cambridge,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cambridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cambridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [15]:
boston_onehot.shape

(232, 102)

In [16]:
boston_grouped = boston_onehot.groupby('Neighborhood').mean().reset_index()
boston_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Beer Garden,Belgian Restaurant,Bookstore,Boutique,Burger Joint,Burrito Place,Bus Station,Café,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Coffee Shop,College Theater,Comedy Club,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Dry Cleaner,Falafel Restaurant,Farmers Market,Fish Market,Food & Drink Shop,Food Truck,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym / Fitness Center,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Lingerie Store,Liquor Store,Mac & Cheese Joint,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,New American Restaurant,Nightclub,Noodle House,Park,Pastry Shop,Pharmacy,Pilates Studio,Pizza Place,Plaza,Pub,Ramen Restaurant,Record Shop,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Ski Chalet,Smoke Shop,Spa,Steakhouse,Street Food Gathering,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Train,Vegetarian / Vegan Restaurant,Wine Shop
0,Boston,0.010000,0.000000,0.000000,0.00000,0.000000,0.00000,0.01,0.070000,0.000000,0.020000,0.01,0.01,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.000000,0.01,0.01,0.01,0.030000,0.000000,0.01,0.01,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.01,0.01,0.01,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.01,0.020000,0.010000,0.000000,0.04,0.000000,0.020000,0.010000,0.000000,0.01,0.16,0.01,0.01,0.01,0.00000,0.01,0.01,0.01,0.000000,0.00000,0.00000,0.00000,0.000000,0.01,0.000000,0.000000,0.00000,0.020000,0.01,0.000000,0.00000,0.020000,0.01,0.030000,0.000000,0.01,0.02,0.010000,0.030000,0.09000,0.000000,0.000000,0.01,0.000000,0.000000,0.01,0.01,0.000000,0.000000,0.00000,0.000000,0.010000,0.000000,0.000000,0.01,0.00000,0.010000,0.010000
1,Brighton,0.000000,0.000000,0.021739,0.00000,0.000000,0.00000,0.00,0.065217,0.065217,0.021739,0.00,0.00,0.00,0.000000,0.00,0.000000,0.021739,0.021739,0.021739,0.043478,0.00,0.00,0.00,0.065217,0.000000,0.00,0.00,0.000000,0.000000,0.021739,0.021739,0.000000,0.021739,0.043478,0.00,0.00,0.00,0.021739,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.021739,0.021739,0.021739,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,0.021739,0.00000,0.00000,0.00000,0.021739,0.00,0.000000,0.000000,0.00000,0.043478,0.00,0.021739,0.00000,0.065217,0.00,0.065217,0.000000,0.00,0.00,0.021739,0.021739,0.00000,0.000000,0.021739,0.00,0.021739,0.021739,0.00,0.00,0.021739,0.021739,0.00000,0.021739,0.021739,0.000000,0.000000,0.00,0.00000,0.000000,0.021739
2,Cambridge,0.045455,0.000000,0.000000,0.00000,0.045455,0.00000,0.00,0.022727,0.000000,0.022727,0.00,0.00,0.00,0.022727,0.00,0.045455,0.000000,0.022727,0.022727,0.022727,0.00,0.00,0.00,0.022727,0.022727,0.00,0.00,0.022727,0.022727,0.022727,0.000000,0.022727,0.022727,0.000000,0.00,0.00,0.00,0.000000,0.00,0.022727,0.068182,0.022727,0.022727,0.00,0.000000,0.000000,0.000000,0.00,0.022727,0.022727,0.000000,0.022727,0.00,0.00,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,0.000000,0.00000,0.00000,0.00000,0.000000,0.00,0.022727,0.022727,0.00000,0.000000,0.00,0.022727,0.00000,0.068182,0.00,0.000000,0.022727,0.00,0.00,0.000000,0.045455,0.00000,0.022727,0.000000,0.00,0.000000,0.045455,0.00,0.00,0.000000,0.045455,0.00000,0.000000,0.000000,0.022727,0.000000,0.00,0.00000,0.022727,0.000000
3,Jamaica Plain,0.047619,0.047619,0.023810,0.02381,0.000000,0.02381,0.00,0.047619,0.023810,0.000000,0.00,0.00,0.00,0.047619,0.00,0.000000,0.023810,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.071429,0.000000,0.00,0.00,0.000000,0.000000,0.023810,0.000000,0.0

This dataframe shows different services of each areas. Only Boston and Cambridge has just a couple vegan restaurants.

In [17]:
boston_grouped.shape

(4, 102)

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = boston_grouped['Neighborhood']

for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Boston,Italian Restaurant,Seafood Restaurant,Bakery,Historic Site,Sandwich Place,Pub,Coffee Shop,Greek Restaurant,Pizza Place,Bar
1,Brighton,Pizza Place,Bakery,Bank,Pub,Coffee Shop,Park,Dry Cleaner,Chinese Restaurant,Pharmacy,Bus Station
2,Cambridge,Pizza Place,Furniture / Home Store,Sushi Restaurant,Burger Joint,Sandwich Place,Spa,Yoga Studio,Art Museum,Vegetarian / Vegan Restaurant,Coffee Shop
3,Jamaica Plain,Coffee Shop,Ice Cream Shop,Pizza Place,Accessories Store,Bakery,Bookstore,Park,Yoga Studio,Thrift / Vintage Store,Taco Place


This dataframe is most markable to show how great needs is to set up more Vegan restaurants, demand is quite high and supply is such of weak.

In [21]:
# set number of clusters
kclusters = 4

boston_grouped_clustering = boston_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([0, 3, 2, 1], dtype=int32)

In [33]:
# add clustering labels

boston_merged = boston_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
boston_merged = boston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

boston_merged.head() # check the last columns!


,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cambridge,42.373611,-71.110558,Pizza Place,Furniture / Home Store,Sushi Restaurant,Burger Joint,Sandwich Place,Spa,Yoga Studio,Art Museum,Vegetarian / Vegan Restaurant,Coffee Shop
1,Boston,42.361145,-71.057083,Italian Restaurant,Seafood Restaurant,Bakery,Historic Site,Sandwich Place,Pub,Coffee Shop,Greek Restaurant,Pizza Place,Bar
2,Brighton,42.347800,-71.156600,Pizza Place,Bakery,Bank,Pub,Coffee Shop,Park,Dry Cleaner,Chinese Restaurant,Pharmacy,Bus Station
3,Jamaica Plain,42.312600,-71.111500,Coffee Shop,Ice Cream Shop,Pizza Place,Accessories Store,Bakery,Bookstore,Park,Yoga Studio,Thrift / Vintage Store,Taco Place


Map show how centralized these services are. Question is there any needs to set up restaurant outside of these most common areas.
Answer for this needs more local information what is not able to get public data.

In [39]:
latitude=42.347800  #boston_merged[2, 'Latitude']
longitude=-71.156600  #boston_merged[2, 'Longitude']
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map    Venue Latitude	Venue Longitude	Venue Category
markers_colors = []
cluster=1
# for lat, lon, poi, cluster in zip(boston_merged['Latitude'], boston_merged['Longitude'], boston_merged['Neighborhood'], boston_merged['Cluster Labels']):
for lat, lon, poi in zip(boston_venues['Venue Latitude'], boston_venues['Venue Longitude'], boston_venues['Neighborhood']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters